# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from scipy import optimize, special

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
#import inauguralproject


# Question 1

**Explain how you solve the model**

In [ ]:

import HouseholdSpecializationModel
model = HouseholdSpecializationModel.HouseholdSpecializationModelClass()

alpha_range = np.array([0.25, 0.50, 0.75])
sigma_range = np.array([0.5, 1.0, 1.5])

# create meshgrid of alpha and sigma values
alpha, sigma = np.meshgrid(alpha_range, sigma_range)

# calculate HF/HM ratio for each combination of alpha and sigma
hf_hm_ratio = np.zeros_like(alpha)
for i in range(alpha.shape[0]):
    for j in range(alpha.shape[1]):
        # set alpha and sigma values for model instance
        model.par.alpha = alpha[i, j]
        model.par.sigma = sigma[i, j]
        # solve model and calculate HF/HM ratio
        opt = model.solve_discrete()
        hf_hm_ratio[i, j] = opt.HF / opt.HM

# plot the surface

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(1,1,1,projection='3d')
ax.plot_surface(alpha, sigma, hf_hm_ratio,cmap=cm.coolwarm,linewidth=0,antialiased=False)

ax.set_xlabel('alpha')
ax.set_ylabel('sigma')
ax.set_zlabel('HF/HM ratio', rotation=90)

ax.invert_xaxis()
#show me the text on z axis
#make the graph wider so i can see the text on the z-axis
fig.set_size_inches(10, 8)
ax.zaxis.labelpad=-0.7
#make the graph wider
#rotate the graph
#ax.view_init(5, 88)




In [ ]:
import numpy as np

# define alpha and sigma ranges
alpha_range = np.array([0.25, 0.50, 0.75])
sigma_range = np.array([0.5, 1.0, 1.5])

# create meshgrid of alpha and sigma values
alpha, sigma = np.meshgrid(alpha_range, sigma_range)

# calculate HF/HM ratio for each combination of alpha and sigma
hf_hm_ratio = np.zeros_like(alpha)
for i in range(alpha.shape[0]):
    for j in range(alpha.shape[1]):
        # set alpha and sigma values for model instance
        model.par.alpha = alpha[i, j]
        model.par.sigma = sigma[i, j]
        # solve model and calculate HF/HM ratio
        opt = model.solve_discrete()
        hf_hm_ratio[i, j] = opt.HF / opt.HM

# print table header
print("| alpha | sigma | HF/HM ratio |")
print("|-------|-------|-------------|")

# print table rows
for i in range(alpha_range.shape[0]):
    for j in range(sigma_range.shape[0]):
        print("|  {:.2f} |  {:.1f} |     {:.4f} |".format(alpha[i, j], sigma[i, j], hf_hm_ratio[i, j]))


# Question 2

Explain your code and procedure

In [ ]:
# solve for each wF value
for i, wF in enumerate(model.par.wF_vec):
    model.par.wF = wF
    opt = model.solve_discrete()
    model.sol.LM_vec[i] = opt.LM
    model.sol.HM_vec[i] = opt.HM
    model.sol.LF_vec[i] = opt.LF
    model.sol.HF_vec[i] = opt.HF

# calculate HF/HM
HF_HM = model.sol.HF_vec / model.sol.HM_vec

# plot log(HF/HM) against log(wF/wM)
plt.plot(np.log(model.par.wF_vec/model.par.wM), np.log(HF_HM))
plt.xlabel('log(wF/wM)')
plt.ylabel('log(HF/HM)')
plt.show()


# Question 3

Explain your code and procedure

In [ ]:
import HouseholdSpecializationModel2
model2 = HouseholdSpecializationModel2.HouseholdSpecializationModelClass2()

# solve for each wF value
for i, wF in enumerate(model2.par.wF_vec):
    model2.par.wF = wF
    opt = model2.solve_discrete()
    model2.sol.LM_vec[i] = opt.LM
    model2.sol.HM_vec[i] = opt.HM
    model2.sol.LF_vec[i] = opt.LF
    model2.sol.HF_vec[i] = opt.HF

# calculate HF/HM
HF_HM = model2.sol.HF_vec / model2.sol.HM_vec

# plot log(HF/HM) against log(wF/wM)
plt.plot(np.log(model2.par.wF_vec/model2.par.wM), np.log(HF_HM))
plt.xlabel('log(wF/wM)')
plt.ylabel('log(HF/HM)')
plt.show()


ADD CONCISE CONLUSION.

# Question 4

In [ ]:
def ssd(alpha, sigma, model):
    """Calculate sum of squared differences between model and regression coefficients"""
    
    # Set alpha and sigma
    model.par.alpha = alpha
    model.par.sigma = sigma
    
    # Solve model for vector of female wages
    model.solve_wF_vec(discrete=True)
    
    # Run regression
    model.run_regression()
    
    # Calculate sum of squared differences
    ssd = (model.sol.beta0 - model.par.beta0_target)**2 + (model.sol.beta1 - model.par.beta1_target)**2
    
    return ssd



In [ ]:

# Define function to minimize
obj_func = lambda x: ssd(x[0], x[1], model)

# Set initial guess
x0 = [0.5, 1.0]

# Minimize function
res = optimize.minimize(obj_func, x0)

# Extract optimal values of alpha and sigma
alpha_hat = res.x[0]
sigma_hat = res.x[1]


In [33]:
import numpy as np
from scipy.optimize import minimize

# Define the function to be minimized
def objective(x):
    beta0_hat = 0.4 - x[0]
    beta1_hat = -0.1 - x[1]
    return beta0_hat**2 + beta1_hat**2

# Set the initial guess for alpha and sigma
x0 = np.array([0.5, 0.5])

# Define the bounds for alpha and sigma
bounds = ((0.001, 0.999), (0.001, 0.999))

# Use the Nelder-Mead method to minimize the objective function
result = minimize(objective, x0, method='nelder-mead', bounds=bounds)

# Print the optimal values of alpha and sigma
print("Optimal values:")
print("alpha =", result.x[0])
print("sigma =", result.x[1])

Optimal values:
alpha = 0.40001156250946246
sigma = 0.001


TypeError: cannot unpack non-iterable NoneType object